# Independnet Study Week 3 - Clustering / Text Processing

## Preperation

In [1]:
import numpy as np
import igraph
import pickle
import requests
import boilerpipe
import opengraph
import pprint
import csv

# Hide some silly output
import logging
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Import everything we need from graphlab
import graphlab as gl

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Part 1 - Heirarchical Clustering

The only algo in the readings for clustering is (agglomerative) heirarchical clustering, which I attempt to implement here.

In [69]:
clusterGraph = igraph.Graph()

# List of vertex names
vertices = range(10)

# Add vertices to the graph
clusterGraph.add_vertices(len(vertices))
clusterGraph.vs["name"] = vertices

# Setup edges
edges = [ (0, 1), (0,2), (1, 2), (2, 3), (4,0), (3, 4), (1, 4), (4, 5), (5, 6), (6, 7), (7,8), (6,8), (6, 9), (8, 9), (7,9) ]
clusterGraph.add_edges( edges )

# Display (popup, since Cairo is a pain to install)
layout = clusterGraph.layout("graphopt")
igraph.plot(clusterGraph, layout = layout, vertex_label=vertices)

In [41]:
distances = clusterGraph.shortest_paths_dijkstra()
distances

[[0, 1, 1, 2, 1, 2, 3, 4, 4, 4],
 [1, 0, 1, 2, 1, 2, 3, 4, 4, 4],
 [1, 1, 0, 1, 2, 3, 4, 5, 5, 5],
 [2, 2, 1, 0, 1, 2, 3, 4, 4, 4],
 [1, 1, 2, 1, 0, 1, 2, 3, 3, 3],
 [2, 2, 3, 2, 1, 0, 1, 2, 2, 2],
 [3, 3, 4, 3, 2, 1, 0, 1, 1, 1],
 [4, 4, 5, 4, 3, 2, 1, 0, 1, 1],
 [4, 4, 5, 4, 3, 2, 1, 1, 0, 1],
 [4, 4, 5, 4, 3, 2, 1, 1, 1, 0]]

In [45]:
#dendrogram = igraph_kite.community_edge_betweenness()
#igraph.plot(dendrogram)

from collections import defaultdict
from scipy.cluster import hierarchy
from scipy.spatial import distance

t = 1.0
Y = distance.squareform(distances)
Z = hierarchy.complete(Y)  # Creates HC using farthest point linkage

labels = clusterGraph.vs["name"]

# This partition selection is arbitrary, for illustrive purposes
membership = list(hierarchy.fcluster(Z, t=t))
membership

# Create collection of lists for blockmodel
partition=defaultdict(list)
for n,p in zip(list(range(len(membership))), membership):
    partition[p].append(labels[n])
pprint.pprint(dict(partition))

{1: [6, 7, 8, 9], 2: [4, 5], 3: [2, 3], 4: [0, 1]}


In [290]:
# Custom example
testVertices = [0, 1, 2, 3]
testGraph = igraph.Graph()
testGraph.add_vertices(len(testVertices))
testGraph.vs["name"] = testVertices

# Setup edges
testGraph.es["weight"] = 1.0
testEdges = [ (0, 1), (1, 2), (2, 3), (0, 2) ]
testGraph.add_edges( testEdges )
testGraph[0, 1] = 1
testGraph[1, 2] = 3
testGraph[2, 3] = 1
testGraph[0, 2] = 5

# Display (popup, since Cairo is a pain to install)
def plotGraph(testGraph):
    layout = testGraph.layout("graphopt")
    visual_style = {}
    visual_style["vertex_label"] = testGraph.vs["name"]
    visual_style["vertex_size"] = 40
    visual_style["margin"] = 20
    visual_style["edge_width"] = [10 - edge["weight"] for edge in testGraph.es]
    igraph.plot(testGraph, layout = layout, **visual_style)
plotGraph(testGraph)

clusterNum = len(testVertices)
clustersHist = [ {v:v for v in testVertices} ]
nodesToCluster = testVertices[:]

distHist = [ ]
while True:
    # Calculate shortest distance matrix from current state of graph
    distances = testGraph.shortest_paths_dijkstra(weights="weight")
    print "Distances: "
    pprint.pprint(distances)
    distHist.append(distances)
    
    # Are we done?
    if len(distances) == 1:
        break
    
    # PROBLEM FOR TOMORROW: constant issues when using names versus ids. not sure how to cope
    nodesToCluster = [v.index for v in testGraph.vs ]
    
    # Vertex to cluster and the closest vertex to it
    nodeToCluster = nodesToCluster.pop(0)
    nodeToClusterWith = distances[nodeToCluster].index(sorted(distances[nodeToCluster])[1])
    
    # Name version of above vertices
    nodeToCluster_name = testGraph.vs[nodeToCluster]['name']
    nodeToClusterWith_name = testGraph.vs[nodeToClusterWith]['name']

    # Add new node for the cluster
    testGraph.add_vertex(name=clusterNum)
    
    # IF there is an edge between cluster nodes, makes next calculation easier
    testGraph.delete_edges(testGraph.es.select(_source=nodeToCluster, _target=nodeToClusterWith))  
    
    # Replace all old edges to the clustered node #1
    for edge in testGraph.incident(nodeToCluster, mode="in"):
        #print edge, testGraph.es[edge].attributes()['weight']
        #print (clusterNum, testGraph.es[edge].target)
        testGraph.add_edges([(testGraph.vs.select(name=clusterNum)[0], testGraph.es[edge].target)])
        testGraph[testGraph.vs.select(name=clusterNum)[0], testGraph.es[edge].target] = testGraph.es[edge].attributes()['weight']

# TODO: fix this, need to get minimum weight
#     for edge in testGraph.incident(nodeToClusterWith, mode="in"):
#         #print edge, testGraph.es[edge].attributes()['weight']
#         #print (clusterNum, testGraph.es[edge].target)
#         testGraph.add_edges([(clusterNum, testGraph.es[edge].target)])
#         testGraph[clusterNum, testGraph.es[edge].target] = testGraph.es[edge].attributes()['weight']
#         print len(testGraph.es)

    # Delete the nodes that were clustered
    testGraph.delete_vertices([nodeToCluster, nodeToClusterWith])
    
    # Show new graph again
    plotGraph(testGraph)
    
    # Increase to get next cluster name
    clusterNum += 1

Distances: 
[[0.0, 1.0, 4.0, 5.0],
 [1.0, 0.0, 3.0, 4.0],
 [4.0, 3.0, 0.0, 1.0],
 [5.0, 4.0, 1.0, 0.0]]
Distances: 
[[0.0, 1.0, 5.0], [1.0, 0.0, 6.0], [5.0, 6.0, 0.0]]
Distances: 
[[0.0, 5.0], [5.0, 0.0]]
Distances: 
[[0.0]]


## Part 2  - Text Processing

Continuing from last week, I extend the textual analysis to the articles linked to tweets from the think-tank accounts. Lets start off importing statuses from pickle object, listing out our accounts and extracting all the urls we can get.

In [68]:
# Open content and accounts from previous week
allStatuses = pickle.load( open( "../Week2/allStatuses", "rb" ) )
accounts = allStatuses.keys()

# Extract all the links from the crawled tweets
links = { }
for account in accounts:
    links[account] = [url.expanded_url  for status in allStatuses[account] for url in status.urls ]

Lets look at how boilerpipe works for a single article:

In [110]:
# Boiler pipe for a single article example
from boilerpipe.extract import Extractor
extractor = Extractor(extractor='ArticleExtractor', url=links['AccuracyInMedia'][0])
extracted_text_default = extractor.getText()
print extracted_text_default

Home Faculty Lounge Rachel Dolezal Let Go at Eastern Washington University
Rachel Dolezal Let Go at Eastern Washington University
June 25, 2015
,  Spencer Irvine, Leave a comment
The controversial adjunct professor and former head of the Spokane, Washington NAACP chapter will not be retained, in all likelihood. But, her past is more head-scratching than her recent comments on her being African-American.
Book Review
Have We Lost the Cultural War?
Paul Kengor’s new book, Takedown: From Communists to Progressives, How the Left Has Sabotaged Family and Marriage, provides a detailed explanation of why, according to a new Gallup poll, “Americans are more likely now than…
, Spencer Irvine , No Comment
Coming to a university near you
Before you find him on offer as a university speaker or course, you may want to read the meticulously documented story of Cop killer Mumia Abu-Jamal by former Accuracy in Academia executive director Dan Flynn.
© Accuracy In Academia
Receive our FREE email newslett

As you can see, the results are okay. There are different kinds of extractors, but this one seems to do well enough. Boilerpipe does better with articles from FAIR. Let us look at how we could use opengraph to extract information (if boilerpipe doesn't work well)

In [76]:
for url in links["AccuracyInMedia"][:5]:
    og =  opengraph.OpenGraph(url=url, scrape=True)
    print "%s. %s" % (og.title, og.description)
    print

Rachel Dolezal Let Go at Eastern Washington University. The controversial adjunct professor and former head of the Spokane, Washington NAACP chapter will not be retained, in all likelihood. But, her past is more head-scratching than her recent comments ...

Supreme Court Upholds ObamaCare, Again, in King v. Burwell  . Yup, by a 6-3 vote and the majority written by Chief Justice Roberts, ObamaCare’s illegal subsidies are ruled legal.

IRS shelled out $18.8 Million to Contractors who didn’t pay Back Taxes  . Way to protect American taxpayer dollars, IRS. “The IRS granted 57 contracts worth $18.8 million to corporations with unpaid back taxes from 2012 to 2013, according to a March 26 Treasury Inspector General for Tax Administration (TIGTA) report.”

Study finds less than 5% of Colleges Protect 1st Amendment Rights  . This isn’t shocking news as colleges like to impose speech codes and free speech zones on their campuses: “A new study says that less than 5 percent of colleges and univers

This looks a little better, though it is not much more data. Lets use it and crawl the links for all accounts using BoilerPipe and OpenGraph as a backup in case of an error:

In [104]:
content = {}
LIMIT = 50

# Try to open content from pickle file
try:
    content = pickle.load(open("crawledContent.pkl", "rb"))
except Exception as e:
    # Otherwise, loop thru all acounts
    for account in accounts:
        # Init account to have no content
        content[account] = {}
        print "Starting account %s" % account

        # Loop thru all links for the account
        for url in links[account][:LIMIT]:
            # Attempt to use boilerpipe's article extractor on url
            try:
                print " -- extracting %s with boilerpipe" % url
                extractor = Extractor(extractor='ArticleExtractor', url=url)
                content[account][url] = extractor.getText()
            except Exception as e:
                # If there is any issue, try using OpenGraph metadata
                try:
                    print " -- failed; extracting %s with og" % url
                    og =  opengraph.OpenGraph(url=url, scrape=True)
                    content[account][url] = "%s. %s" % (og.title, og.description)
                except Exception as e:
                    pass
    
    # Save results
    pickle.dump(content, open("crawledContent.pkl", "wb"))

Starting account ips_dc
 -- extracting http://www.ips-dc.org/?p=38436 with boilerpipe
 -- extracting http://fpif.org/our-refugee-world with boilerpipe
 -- extracting http://www.ips-dc.org/francis-v-washington/ with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38416 with boilerpipe
 -- extracting http://bit.ly/1L3dMtX with boilerpipe
 -- extracting http://bit.ly/1BAKYqo with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38414 with boilerpipe
 -- extracting http://bit.ly/1IyIUfP with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38412 with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38410 with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38410 with boilerpipe
 -- extracting http://bit.ly/1LiZ565 with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38399 with boilerpipe
 -- extracting http://www.ips-dc.org/hacked/ with boilerpipe
 -- extracting http://www.ips-dc.org/?p=38262 with boilerpipe
 -- extracting http://ips-dc.org/?p=38392 with boilerpipe
 -- extra

Now that we have crawled some data, lets process it to find some collocations (with bigrams)

In [115]:
import string

import nltk
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

def preProcess(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = [w for w in tokens if not w in stopwords.words('english') and not w.isdigit()]
    return " ".join(filtered_words)

def displayBigrams(contentDict, threshold=5):
    tokens = nltk.wordpunct_tokenize(preProcess("".join([ contentDict[url] for url in contentDict])))
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(threshold)
    scored = finder.score_ngrams(bigram_measures.raw_freq)
    return sorted([ (bigram, score) for (bigram, score) in scored ], key=lambda t: t[1], reverse=True)

Lets create a list of bigrams from AccuracyInMedia:

In [116]:
displayBigrams(content['AccuracyInMedia'])

[((u'spencer', u'irvine'), 0.016249533059394843),
 ((u'share', u'author'), 0.0056032872618602915),
 ((u'aim', u'running'), 0.005042958535674262),
 ((u'author', u'spencer'), 0.005042958535674262),
 ((u'blog', u'target'), 0.005042958535674262),
 ((u'brigham', u'young'), 0.005042958535674262),
 ((u'currently', u'works'), 0.005042958535674262),
 ((u'graduated', u'brigham'), 0.005042958535674262),
 ((u'home', u'blog'), 0.005042958535674262),
 ((u'international', u'relations'), 0.005042958535674262),
 ((u'irvine', u'graduated'), 0.005042958535674262),
 ((u'irvine', u'june'), 0.005042958535674262),
 ((u'irvine', u'spencer'), 0.005042958535674262),
 ((u'media', u'related'), 0.005042958535674262),
 ((u'operations', u'social'), 0.005042958535674262),
 ((u'related', u'posts'), 0.005042958535674262),
 ((u'relations', u'currently'), 0.005042958535674262),
 ((u'running', u'operations'), 0.005042958535674262),
 ((u'social', u'media'), 0.005042958535674262),
 ((u'university', u'international'), 0.0050

Comapring it with fairmediawatch:

In [117]:
displayBigrams(content['fairmediawatch'])

[((u'new', u'york'), 0.0037562162734102213),
 ((u'york', u'times'), 0.0026981271823087504),
 ((u'corporate', u'media'), 0.001481324727542059),
 ((u'right', u'wing'), 0.0012168024547666914),
 ((u'united', u'states'), 0.0011638980002116178),
 ((u'women', u'sports'), 0.0011109935456565443),
 ((u'white', u'supremacist'), 0.0010051846365463973),
 ((u'jim', u'naureckas'), 0.0009522801819913237),
 ((u'washington', u'post'), 0.0009522801819913237),
 ((u'even', u'though'), 0.0008993757274362501),
 ((u'op', u'ed'), 0.0008993757274362501),
 ((u'isis', u'propaganda'), 0.0008464712728811766),
 ((u'dylann', u'roof'), 0.0007935668183261031),
 ((u'bernie', u'sanders'), 0.0007406623637710295),
 ((u'communication', u'effective'), 0.000687757909215956),
 ((u'fox', u'news'), 0.000687757909215956),
 ((u'news', u'media'), 0.000687757909215956),
 ((u'remember', u'respectful'), 0.000687757909215956),
 ((u'respectful', u'communication'), 0.000687757909215956),
 ((u'de', u'blasio'), 0.0006348534546608824),
 ((u